# IRF Handling

This notebook describes the handling of the uw.irfs module for loading IRF information. 


## Setup


In [89]:
import os
import pprint

import numpy as np
import pylab as plt
from astropy.io import fits

import skymaps
from uw.like2 import configuration, bands, dataset
from uw.irfs import caldb,psf,effective_area,exposure, irfman

map(reload,(configuration,bands,dataset,caldb,
            psf,effective_area,exposure,irfman))
os.environ['CALDB'] = os.path.expandvars('${BASE_DIR}/irfs/caldb/CALDB')
os.environ['TIMING_DIR'] = os.path.expandvars('${GLAST_EXT}/extFiles/v0r9/jplephem')
cwd = "/tmp/wallacee/response"
if not os.path.exists(cwd):
    if not os.path.exists(os.path.dirname(cwd)):
        os.mkdir(os.path.dirname(cwd))
    os.mkdir(cwd)
os.chdir(cwd)



## CALDB

The CALDB class manages reading in info from a CALDB database. It takes one optional argument, CALDB_dir, specifying the path to the base of the CALDB directory structure. If not given, it is read from the $CALDB environment variable. 

In [90]:
cdb = caldb.CALDB()

A CALDB object can be called with a specific irf ('psf','aeff', or 'edisp'), version, event class, and event type. The return value is a dictionary containing the filenames and irf extensions for each event type matching the given version selection. If multiple event types are requested, the return value is a dict in which the keys are the (integer) event types and the values are dicts of the same form as would be returned for a single type. 

### A note about event type specifications

In the CALDB class, and elsewhere in the uw.irfs module, any function that takes an argument specifying an event type will (or at least should) accept either an integer from 0-9, a string, or a sequence of either. Integer event types correspond to the  appropriate bit in the collaboration-standard event type bitmask: (0=back, 1=front, 2=psf0, ..., 5=psf3, 6=edisp0,...). Strings can be either "front", "back", "psf#", or "edisp#", and are case-insensitive. Additionally, the strings "fb", "psf", and "edisp" can be used to indicate all types from the given partition (i.e., "fb" is equivalent to ("front","back"), and so on). 

Where event type specifications are stored internally or returned from functions, the integer form is generally (hopefully consistently). 

TODO: encapsulate the event type handling in a more transparent and less error-prone form.

In [91]:
psf3 = cdb('psf',version='P8R2_V6',event_class='source',event_type='psf3')
print("PSF files for P8R2_SOURCE_V6::PSF3 :\n")
pprint.pprint(psf3)

psf_all = cdb('psf',version='P8R2_V6',event_class='source',event_type='psf')
print("PSF files for P8R2_SOURCE_V6::PSF :\n")
pprint.pprint(psf_all)


PSF files for P8R2_SOURCE_V6::PSF3 :

{'extensions': {'FISHEYE_CORR': 12, 'PSF_SCALING': 11, 'RPSF': 10},
 'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-01-01/irfs/caldb/CALDB/data/glast/lat/bcf/psf/psf_P8R2_SOURCE_V6_PSF.fits'}
PSF files for P8R2_SOURCE_V6::PSF :

{2: {'extensions': {'FISHEYE_CORR': 3, 'PSF_SCALING': 2, 'RPSF': 1},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-01-01/irfs/caldb/CALDB/data/glast/lat/bcf/psf/psf_P8R2_SOURCE_V6_PSF.fits'},
 3: {'extensions': {'FISHEYE_CORR': 6, 'PSF_SCALING': 5, 'RPSF': 4},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-01-01/irfs/caldb/CALDB/data/glast/lat/bcf/psf/psf_P8R2_SOURCE_V6_PSF.fits'},
 4: {'extensions': {'FISHEYE_CORR': 9, 'PSF_SCALING': 8, 'RPSF': 7},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc4

## Effective Area

The effective_area module provides the EffectiveArea class, which is currently a simple wrapper around skymaps.EffectiveArea. 

In [92]:
aeff_info = cdb('aeff',version='P8R2_V6',event_class='source',event_type='psf')
print("Effective area files for P8R2_SOURCE_V6::PSF :\n")
pprint.pprint(aeff_info)
aeff_psf3 = aeff_info[5]
aeff = effective_area.EffectiveArea(aeff_psf3['filename'],
             aeff_extension = aeff_psf3['extensions']['EFF_AREA'],
             eff_params_extension=aeff_psf3['extensions']['EFFICIENCY_PARS'])

Effective area files for P8R2_SOURCE_V6::PSF :

{2: {'extensions': {'EFFICIENCY_PARS': 3, 'EFF_AREA': 1, 'PHI_DEP': 2},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-01-01/irfs/caldb/CALDB/data/glast/lat/bcf/ea/aeff_P8R2_SOURCE_V6_PSF.fits'},
 3: {'extensions': {'EFFICIENCY_PARS': 6, 'EFF_AREA': 4, 'PHI_DEP': 5},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-01-01/irfs/caldb/CALDB/data/glast/lat/bcf/ea/aeff_P8R2_SOURCE_V6_PSF.fits'},
 4: {'extensions': {'EFFICIENCY_PARS': 9, 'EFF_AREA': 7, 'PHI_DEP': 8},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-01-01/irfs/caldb/CALDB/data/glast/lat/bcf/ea/aeff_P8R2_SOURCE_V6_PSF.fits'},
 5: {'extensions': {'EFFICIENCY_PARS': 12, 'EFF_AREA': 10, 'PHI_DEP': 11},
     'filename': '/nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Opti

## Exposure

The exposure.Exposure class handles the combination of the effective area and the livetime. Currently a wrapper around the C++ implementation, mostly just copied from like2.exposure. 

In [93]:
#DataSet for access to the livetime
dataspec=dict(
    ft1files=os.path.expandvars('$FERMI/data/P8_P302/zmax105/P302_Source_001_zmax105.fits'),
    ft2files=os.path.expandvars('/afs/slac/g/glast/groups/catalog/P8_P302/ft2_2008.fits'),
    binfile=os.path.join(cwd,'P302_4bpd_psf.fits'),
    ltcube=os.path.join(cwd,'ltcube.fits'),
    binsperdec=4,
    psf_event_types=True
)
dset = dataset.DataSet(dataspec,irf='P8R2_SOURCE_V6')

exp = exposure.Exposure(dset.lt,aeff,cthetamin=np.cos(np.radians(dset.theta_cut.get_bounds()[1])))

ens = np.logspace(2,5,13)
st = "Exposure at ra,dec = (0,0), energy={}: {}"
sd = skymaps.SkyDir(0,0)
for e in ens:
    print(st.format(e,exp(sd,e)))

processing cuts:  ZENITH_ANGLE zenith_cut
ft1_cut DSTYP4: ZENITH_ANGLE
DSUNI4: deg
DSVAL4: 0:105
DSREF4: None
processing cuts:  THETA theta_cut
ft1_cut None
processing cuts:  EVENT_CLASS event_class_cut
ft1_cut DSTYP1: BIT_MASK(EVENT_CLASS,128,P8R2)
DSUNI1: DIMENSIONLESS
DSVAL1: 1:1
DSREF1: None
File /tmp/wallacee/response/P302_4bpd_psf.fits not found
using Gti for creating binned photon file Gti: 416 intervals from 239557417 to 241960000, on time 1975018
Creating binfile from 1 FT1 files


/afs/slac/g/glast/groups/catalog/python/anaconda/lib/python2.7/site-packages/pyfits/file.py:339: UserWarning: Overwriting existing file '/tmp/wallacee/response/P302_4bpd_psf.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)


checking ltcube: failed clobber on /tmp/wallacee/response/ltcube.fits
on iteration 0


/afs/slac/g/glast/groups/catalog/python/anaconda/lib/python2.7/site-packages/pyfits/file.py:339: UserWarning: Overwriting existing file '/tmp/wallacee/response/ltcube.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)


Exposure at ra,dec = (0,0), energy=100.0: 250049787.292
Exposure at ra,dec = (0,0), energy=177.827941004: 426361955.44
Exposure at ra,dec = (0,0), energy=316.227766017: 589017690.83
Exposure at ra,dec = (0,0), energy=562.34132519: 755780395.253
Exposure at ra,dec = (0,0), energy=1000.0: 874579141.366
Exposure at ra,dec = (0,0), energy=1778.27941004: 950489217.356
Exposure at ra,dec = (0,0), energy=3162.27766017: 963590645.189
Exposure at ra,dec = (0,0), energy=5623.4132519: 927717503.094
Exposure at ra,dec = (0,0), energy=10000.0: 930566521.764
Exposure at ra,dec = (0,0), energy=17782.7941004: 933555352.031
Exposure at ra,dec = (0,0), energy=31622.7766017: 972056190.086
Exposure at ra,dec = (0,0), energy=56234.132519: 1008206856.48
Exposure at ra,dec = (0,0), energy=100000.0: 987718147.403


The exposure.BandExposure class provides a representation of the exposure at a specific energy.

In [94]:
assert(np.all([exposure.BandExposure(exp,e)(sd)==exp(sd,e) for e in ens]))

## PSF

The psf.PSF class provides a representation of the PSF. The PSF class is callable, with an angular deviation and energy. If exposure weights have been calculated (see below), the returned value is the exposure-weighted average of the PSF over inclination angles. If no exposure has been provided, the returned value is just the on-axis PSF density. 

NOTE: I'm not sure that the PSF normalization is being correctly calculated from the CALDB values.   

In [95]:
mypsf = psf.PSF(psf3['filename'],rpsf_extension = psf3['extensions']['RPSF'],
                psf_scaling_extension = psf3['extensions']['PSF_SCALING'])

st = "On-axis PSF density at delta={} degrees, e=100: = {}"
deltas = np.linspace(0,1,5)
for d in deltas:
    print(st.format(d,mypsf(d,100)))

On-axis PSF density at delta=0.0 degrees, e=100: = 188.235603849
On-axis PSF density at delta=0.25 degrees, e=100: = 0.0221948017143
On-axis PSF density at delta=0.5 degrees, e=100: = 0.000392083899762
On-axis PSF density at delta=0.75 degrees, e=100: = 4.35369395109e-05
On-axis PSF density at delta=1.0 degrees, e=100: = 9.62450301971e-06


A PSF can be initialized with an optional exposure argument. If provided, the given exposure will be used to calculate weights for averaging over inclination angles.

NOTE: The averaging over inclination angles is not implemented yet. Should be there soon! 

In [96]:
mypsf = psf.PSF(psf3['filename'],rpsf_extension = psf3['extensions']['RPSF'],
                psf_scaling_extension = psf3['extensions']['PSF_SCALING'],exposure=exp)

st = "Theta-averaged PSF density at delta={} degrees, e=100: = {}"
deltas = np.linspace(0,1,5)
for d in deltas:
    print(st.format(d,mypsf(d,100)))

Theta-averaged PSF density at delta=0.0 degrees, e=100: = 188.235603849
Theta-averaged PSF density at delta=0.25 degrees, e=100: = 0.0221948017143
Theta-averaged PSF density at delta=0.5 degrees, e=100: = 0.000392083899762
Theta-averaged PSF density at delta=0.75 degrees, e=100: = 4.35369395109e-05
Theta-averaged PSF density at delta=1.0 degrees, e=100: = 9.62450301971e-06


psf.BandPSF provides a PSF representation for a specific energy. A BandPSF for energy e can be produced from a PSF object via the PSF.band_psf method.

In [97]:
bandpsf = mypsf.band_psf(100)

assert(np.all(mypsf(d,100)==bandpsf(d) for d in deltas))

## IrfManager

The uw.irfman module provides the IrfManager class to manage a set of PSF and Exposure objects for a given IRF selection. It is initialized from a dataset, assumed to be specified as a like2.dataset.DataSet. It must at least have attributes "irf_version" (the IRF version, specified with the event class included, e.g. "P8R2_SOURCE_V6"), "lt" (a LivetimeCube object), "psf_event_types" (a boolean indicating whether psf or front/back event types should be used), and "theta_cut" (a uw.data.DSSSimpleRange specifying the theta cut). 

The methods psf(event_type,energy) and exposure(event_type,energy) will return BandPSF and BandExposure objects, respectively, for the given event type and energy. The PSF representations are initialized with the corresponding exposures to ensure appropriate weighting for the averages over theta. 

In [98]:
iman = irfman.IrfManager(dset)
psf = iman.psf(2,100)
exp = iman.exposure(2,100)
print(psf,exp)

(<uw.irfs.psf.BandPSF object at 0x7ff470733890>, <uw.irfs.exposure.BandExposure object at 0x7ff4b1248250>)


A configuration.Configuration initializes an IrfManager to provide the appropriate PSF and Exposure objects. The user interface *should* be compatible with the versions from like2.  

In [99]:

cfg = dict(
    dataspec=dict(
        ft1files=os.path.expandvars('$FERMI/data/P8_P302/zmax105/P302_Source_001_zmax105.fits'),
        ft2files=os.path.expandvars('/afs/slac/g/glast/groups/catalog/P8_P302/ft2_2008.fits'),
        binfile=os.path.join(cwd,'P302_4bpd_fb.fits'),
        ltcube=os.path.join(cwd,'ltcube.fits'),
        binsperdec=4,
    ),
    irf= 'P8R2_SOURCE_V6',
    
    input_model = dict(path='$FERMI/skymodels/P302_7years/uw1002'),

    diffuse = dict(
    ring    = dict(type='HealpixCube', 
            filename='/nfs/slac/g/ki/ki20/elliott/Pass8_GC/gcfit/results/P8_P302_ultraclean_veto_z90/galprop/models/'\
                'diffuse_model_flux_P8_P302_ultraclean_veto_z90_umap_ifilter_Galprop_5rings_IC123_geomLoopI_PS_P8uw963_adaptive_ps_mask_P8uw963_DM_Cusp_n2p5_ISO_plaw_pnorm_psfall.fits',
            correction='galactic_correction_uw1002A.csv', 
            systematic=0.0316), 
    isotrop = dict(type='IsotropicList', filename='isotropic_source_*_4years_P8V3.txt',
            correction='isotropic_correction_*_uw965.csv'),
    limb    = None, 
    SunMoon = 'template_SunMoon_6years_zmax100.fits', 
    ),
    
    extended= 'Extended_archive_v14',
)
with open('config.txt','w') as f:

    f.write(str(cfg))

In [100]:
import os
cwd = os.path.expandvars('/tmp/test_psf_binning')
if not os.path.exists(cwd):
    os.mkdir(cwd)
os.chdir(cwd)
if not os.path.basename(os.getcwd())=='P302_4bpd_fb':
    os.chdir(os.path.join(cwd,'P302_4bpd_fb'))
config = configuration.Configuration(quiet=True)


processing cuts:  ZENITH_ANGLE zenith_cut
ft1_cut DSTYP4: ZENITH_ANGLE
DSUNI4: deg
DSVAL4: 0:105
DSREF4: None
processing cuts:  THETA theta_cut
ft1_cut None
processing cuts:  EVENT_CLASS event_class_cut
ft1_cut DSTYP1: BIT_MASK(EVENT_CLASS,128,P8R2)
DSUNI1: DIMENSIONLESS
DSVAL1: 1:1
DSREF1: None
